In [1]:
!pip install -r requirements.txt

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-_36ndw_z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-_36ndw_z
  Resolved https://github.com/huggingface/transformers to commit d04ec99bec8a0b432fc03ed60cea9a1a20ebaf3c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import torch
import numpy as np

torch.cuda.device_count()

1

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer

tokenizer = LlamaTokenizer.from_pretrained("hf-internal-testing/llama-tokenizer")

In [4]:
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("./chunk_dataset")
val_dataset = load_dataset("IlyaGusev/rulm", split="validation[:10%]")

No config specified, defaulting to: rulm/default
Found cached dataset rulm (/home/jovyan/.cache/huggingface/datasets/IlyaGusev___rulm/default/0.0.1/c94da05286e8839c1219f9a5061c630709a487fb298427a6f5a73ee0fe2cd2cb)


In [5]:
len(dataset[0]['input_ids'])

512

In [6]:
def tokenize(prompt, add_eos_token=True):
    result = tokenizer(prompt)
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    return result

In [7]:
from itertools import chain
def chunk_examples(examples, max_length):
    input_ids_chunks = []
    attention_mask_chunks = []
    # print(len(examples["input_ids"]))
    input_ids = list(chain(*examples["input_ids"]))
    attention_mask = list(chain(*examples["attention_mask"]))
    input_ids_chunks.extend(
        [
            input_ids[i : i + max_length]
            for i in range(0, len(input_ids), max_length)
        ]
    )
    attention_mask_chunks.extend(
        [
            attention_mask[i : i + max_length]
            for i in range(0, len(attention_mask), max_length)
        ]
    )
    return {
        "input_ids": input_ids_chunks,
        "attention_mask": attention_mask_chunks,
    }

In [8]:
def prepare_dataset(dataset):
    tokenized_dataset = dataset.map(lambda x: tokenize(x['text']),
                                    num_proc=16, remove_columns=["meta"])
    chunk_dataset = tokenized_dataset.map(lambda x: chunk_examples(x, 512), batched=True, batch_size=100,
                                          num_proc=16, remove_columns=tokenized_dataset.column_names)
    return tokenized_dataset, chunk_dataset

In [9]:
_, chunk_val_dataset = prepare_dataset(val_dataset)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/IlyaGusev___rulm/default/0.0.1/c94da05286e8839c1219f9a5061c630709a487fb298427a6f5a73ee0fe2cd2cb/cache-f418d707dbed0c01_*_of_00016.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/IlyaGusev___rulm/default/0.0.1/c94da05286e8839c1219f9a5061c630709a487fb298427a6f5a73ee0fe2cd2cb/cache-826a7ac955c9b816_*_of_00016.arrow


In [10]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.unk_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="pt")

In [11]:
from peft import get_peft_model, LoraConfig #, prepare_model_for_int8_training

model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf")
# model = AutoModelForCausalLM.from_pretrained("decapoda-research/llama-7b-hf", load_in_8bit=True, device_map='auto')


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/jovyan/.imgenv-zealous-villani-0/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/jovyan/.imgenv-zealous-villani-0/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/jovyan/.imgenv-zealous-villani-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /home/jovyan/.imgenv-zealous-villani-0 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/jovyan/.imgenv-zealous-villani-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/home/jovyan/.imgenv-zealous-villani-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/jovyan/.imgenv-zealous-villani-0/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The followi

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [12]:
def mem_usage():
    used, total = torch.cuda.mem_get_info()
    free = ((used / 1024) / 1024) / 1024
    total = ((total / 1024) / 1024) / 1024
    print(f"{total - free} gigs out of {total} is used")

In [13]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

6738415616

In [13]:
lc = {
        "r": 8,
        "lora_alpha": 16,
        "lora_dropout": 0.05,
        "bias": "none",
        "target_modules": ["q_proj", "v_proj"],
        "task_type": "CAUSAL_LM"
    }

lora_config = LoraConfig(**lc)
model = get_peft_model(model, lora_config)

In [35]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
params

6738415616

In [63]:
model_parameters = len(list(filter(lambda p: p.requires_grad, model.parameters())))

In [34]:
for n, p in model.named_parameters():
    if p.requires_grad:
        print(n, p.shape)

model.embed_tokens.weight torch.Size([32000, 4096])
model.layers.0.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.k_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.v_proj.weight torch.Size([4096, 4096])
model.layers.0.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.0.mlp.gate_proj.weight torch.Size([11008, 4096])
model.layers.0.mlp.down_proj.weight torch.Size([4096, 11008])
model.layers.0.mlp.up_proj.weight torch.Size([11008, 4096])
model.layers.0.input_layernorm.weight torch.Size([4096])
model.layers.0.post_attention_layernorm.weight torch.Size([4096])
model.layers.1.self_attn.q_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.k_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.v_proj.weight torch.Size([4096, 4096])
model.layers.1.self_attn.o_proj.weight torch.Size([4096, 4096])
model.layers.1.mlp.gate_proj.weight torch.Size([11008, 4096])
model.layers.1.mlp.down_proj.weight torch.Size([4096, 11008])
model

In [26]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=31999)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaR

In [64]:
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
params = sum([*np.prod(p.size()) for p in model_parameters])
params

SyntaxError: iterable unpacking cannot be used in comprehension (2366213081.py, line 2)

In [15]:
mem_usage()

0.404052734375 gigs out of 39.58709716796875 is used


In [16]:
model = model.to("cuda")

In [17]:
import torch
import gc

def get_gpu_objects():
    objs = []

    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
                if str(obj.device) == "cuda:0":
                    objs.append(obj)
        except:
            pass
    return objs

In [18]:
model_objs = get_gpu_objects()

/home/user/conda/lib/python3.9/site-packages/torch/distributed/distributed_c10d.py:284: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(


In [19]:
mem_usage()

26.113037109375 gigs out of 39.58709716796875 is used


In [20]:
bs = 1
X = torch.tensor([dataset[i]["input_ids"] for i in range(bs)])

In [21]:
X.shape

torch.Size([1, 512])

In [22]:
X = X.to("cuda")

In [23]:
model_and_X_objs = get_gpu_objects()
# set(model_and_X_objs) - set(model_objs)

In [29]:
mem_usage()

35.111083984375 gigs out of 39.58709716796875 is used


In [25]:
y = model.forward(X, use_cache=False)

In [27]:
type(model)

transformers.models.llama.modeling_llama.LlamaForCausalLM

In [30]:
y.logits.size()

torch.Size([1, 512, 32000])

In [31]:
after_inference_objs = get_gpu_objects()
new_objs = set(after_inference_objs) - set(model_and_X_objs)

In [33]:
len(new_objs)

774

In [36]:
for no in new_objs:
    print(no.shape)

torch.Size([1, 512, 1])
torch.Size([1, 512, 4096])
torch.Size([1, 32, 512, 128])
torch.Size([1, 512, 4096])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 512, 4096])
torch.Size([1, 512, 4096])
torch.Size([1, 512, 4096])
torch.Size([1, 512, 4096])
torch.Size([1, 512, 11008])
torch.Size([1, 512, 11008])
torch.Size([1, 512, 11008])
torch.Size([1, 512, 11008])
torch.Size([1, 512, 11008])
torch.Size([1, 512, 11008])
torch.Size([1, 512, 11008])
torch.Size([1, 512, 11008])
torch.Size([1, 1, 512, 128])
torch.Size([1, 512, 128])
torch.Size([1, 512, 4096])
torch.Size([1, 32, 512, 512])
torch.Size([1, 512, 1])
torch.Size([1, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 32, 512, 128])
torch.Size([1, 512, 32000])
torch.Size

In [24]:
mem_usage()

33.564208984375 gigs out of 39.58709716796875 is used


In [32]:
y.logits.shape

torch.Size([1, 512, 32000])

In [28]:
y = model.forward(X)

In [48]:
y

CausalLMOutputWithPast(loss=None, logits=tensor([[[-12.7782, -28.6373,   0.9082,  ...,  -7.6488,  -8.5625,  -7.6831],
         [ -7.3367, -16.1066,   0.3829,  ...,  -2.2448,  -3.9622,  -2.7864],
         [ -3.2643,  -4.9375,  -0.5947,  ...,  -0.8401,  -3.1155,  -1.8175],
         ...,
         [ -0.1562,  -3.6233,  12.5931,  ...,   3.1760,   3.3542,   4.4769],
         [ -8.6087, -20.0140,   1.4349,  ...,  -2.2611,  -4.9424,  -1.8318],
         [ -4.8588, -10.9607,   4.6263,  ...,  -1.3441,  -4.0876,  -3.1225]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>), past_key_values=None, hidden_states=None, attentions=None)

In [45]:
mem_usage()

33.564208984375 gigs out of 39.58709716796875 is used


In [25]:
torch.cuda.empty_cache()

In [26]:
mem_usage()

33.542724609375 gigs out of 39.58709716796875 is used


In [37]:
import math

In [43]:
model.base_model.model.model.layers[0].mlp.gate_proj.weight.grad

In [44]:
total_parameters = 0
for n, p in model.named_parameters():
    if p.grad:
        print("kek")
        total_parameters += math.prod([s for s in p.grad])
print(total_parameters)

0


In [48]:
y_params = 0
for i in y.past_key_values:
    for j in i:
        p = 1
        for k in j.shape:
            p *= k
        y_params += p

In [51]:
(((y_params * 32) / 8) / 1024) / 1024

4096.0

In [40]:
y.past_key_values[0][0].shape

torch.Size([8, 32, 512, 128])

In [33]:
y.logits.shape

torch.Size([8, 512, 32000])

In [17]:
    # "trainer": {
    #     "evaluation_strategy": "steps",
    #     "per_device_train_batch_size": 16,
    #     "per_device_eval_batch_size": 16,
    #     "gradient_accumulation_steps": 8,
    #     "eval_steps": 75,
    #     "save_steps": 75,
    #     "logging_steps": 5,
    #     "learning_rate": 0.0003,
    #     "num_train_epochs": 3,
    #     "lr_scheduler_type": "cosine",
    #     "warmup_steps": 50,
    #     "fp16": true,
    #     "bf16": false,
    #     "torch_compile": false,
    #     "optim": "adamw_torch"
    # },
training_args = TrainingArguments(
    evaluation_strategy="steps",
    save_steps=75,
    eval_steps=75,
    warmup_steps=50,
    logging_steps=5,

    gradient_accumulation_steps=8,

    num_train_epochs=3,

    learning_rate=0.0003,
    lr_scheduler_type="cosine",

    per_device_eval_batch_size=2,
    per_device_train_batch_size=2,

    # fp16=True,
    # bf16=False,
    torch_compile=False,
    optim="adamw_torch",

    logging_dir='./logs',
    output_dir='./trained',
    
    no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=chunk_val_dataset,
    data_collator=data_collator
)

In [ ]:
trainer.train()

In [42]:
torch.cuda.empty_cache()

In [29]:
import torch

In [16]:
data = None

In [43]:
data = data_collator([dataset[0], dataset[1]])['input_ids'].to('cuda')

In [44]:
generate_ids = cuda_m(data)

In [64]:
generate_ids.logits.grad

In [45]:
type(generate_ids)

transformers.modeling_outputs.CausalLMOutputWithPast

In [65]:
del generate_ids

In [41]:
del data

In [23]:
generate_ids.keys()

odict_keys(['logits', 'past_key_values'])